In [1]:
from image_compressor.svd_compressor import SVDCompressor
from image_compressor.svd import svd
from image_compressor.compressed import Compressed
from image_compressor.evaluation import mse, psnr, pearson_correlation, cr

In [2]:
# add metrics
def er_score(image, U, singular_values, V, rank):
    original_energy = np.sum(singular_values ** 2)
    current_energy = np.sum(singular_values[:rank] ** 2)
    return current_energy / original_energy

def ssim_score(image, U, singular_values, V, rank):
    L = 255
    k1 = 0.01
    k2 = 0.03
    c1 = (k1 * L) ** 2
    c2 = (k2 * L) ** 2
    c3 = c2 / 2
    
    X = image
    Y = (U[:, :rank] @ np.diag(singular_values[:rank]) @ V[:rank, :]).astype(np.float32)
    mu1, mu2 = np.average(X), np.average(Y)
    luminance = (2 * mu1 * mu2 + c1) / (mu1 ** 2 + mu2 ** 2 + c1)
    std1, std2 = np.std(X), np.std(Y)
    contrast = (2 * std1 * std2 + c2) / (std1 ** 2 + std2 ** 2 + c2)
    covariance = np.matmul((X - mu1).flatten(), (Y - mu2).flatten()) / X.size
    structure = (covariance + c3) / (std1 * std2 + c3)
    return luminance * contrast * structure

In [3]:
# update svd compressor that also take rank and return image, ssim and er
class SVDCompressor_rank():
    def compress(self, image, rank):
        image = image.astype(np.float32)
        m, n = image.shape
        transposed = False
        if n > m:
            image = image.T
            transposed = True
        U, S, V = svd(image)
        if n > m:
            U, V = V.T, U.T

        singular_values = np.diag(S)
        indexes = np.argsort(singular_values)[::-1]
        U = U[:, indexes]
        V = V[indexes, :]
        singular_values = singular_values[indexes]
        if transposed:
            image = image.T

        decompressed = (U[:, :rank] @ np.diag(singular_values[:rank]) @ V[:rank, :]).astype(np.float32)

        return Compressed(U[:, :rank], singular_values[:rank], V[:rank, :]), \
                ssim_score(image, U, singular_values, V, rank), er_score(image, U, singular_values, V, rank)

# Experiments

In [4]:
import numpy as np
import pandas
import cv2

def compress_image(image, rank):
    compressor = SVDCompressor_rank()
    compressed, ssim, er = compressor.compress(image, rank)
    new_image = compressed.to_image()
    return new_image, compressed.get_effective_rank(), mse(image, compressed), psnr(image, compressed),\
            pearson_correlation(image, compressed), cr(image, compressed), ssim, er

In [12]:
image = cv2.imread("image/lenna.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

output = [compress_image(image, rank) for rank in [2**i for i in range(9)]]
table = pandas.DataFrame([i[1:] for i in output], columns = ["Rank", "MSE", "PSNR", "PR", "CR", "SSIM", "ER"])

for i in range(len(output)):
    cv2.imwrite("image/lenna_compressed_{}.jpg".format(output[i][1]), output[i][0])
    
table

,Rank,MSE,PSNR,PR,CR,SSIM,ER
0,1,0.259647,53.986964,0.512934,256.0,0.420913,0.858324
1,2,0.243401,54.267588,0.672492,128.0,0.626216,0.894700
2,4,0.247555,54.194090,0.795645,64.0,0.777468,0.929462
3,8,0.254772,54.069283,0.898880,32.0,0.894773,0.963084
4,16,0.252506,54.108082,0.947998,16.0,0.947039,0.980493
5,32,0.269722,53.821640,0.976512,8.0,0.976433,0.991070
6,64,0.256207,54.044902,0.991576,4.0,0.991614,0.996774
7,128,0.236637,54.389975,0.997965,2.0,0.997991,0.999223
8,256,0.210159,54.905317,0.999689,1.0,0.999705,0.999886
